# 椭圆曲线

常见的椭圆曲线为如下所示的二元三阶方程：$$y^2=x^3+ax+b$$

 ## 有限域上的椭圆曲线

有限域$F_p$的定义：  
1. $F_p$上有$p$（$p$为质数）个元素，分别为$0,1,2,\dots,p-2,p-1$
2. $F_p$上的加法是$a+b\equiv c(mod p)$
3. $F_p$上的乘法是$a\times b\equiv c(mod p)$
4. $F_p$上的除法是$a\div b\equiv c(mod p)，即a\times b^{-1}\equiv c(mod p)$
5. $F_p$上的运算满足交换律、结合律、分配率
6. 如果椭圆曲线上的一点P,存在最小的正整数n，使得数乘$np=\infty$，则称n为P的阶，若n不存在，则称点P是无限阶。  
对于分数的求余，采用逆元将其转换为乘法：$$a/b(modp)=a \times b^{-1}(mod p)$$其中$b^{-1}$是$b$的逆元，满足$b \times b^{-1} \equiv 1(modp)$ 

对于$b \times b^{-1} \equiv 1(modp)$，Simchain采用欧几里德算法求逆元$b^{-1}$

In [1]:
def inv_mod(b, p):
    
    if b < 0 or p <= b:
        b = b % p

    c, d = b, p
    uc, vc, ud, vd = 1, 0, 0, 1
    while c != 0:
        q, c, d = divmod(d, c) + (c,)
        uc, vc, ud, vd = ud - q * uc, vd - q * vc, uc, vc
        
    # 如果d==1，则报错无解    
    assert d == 1
    if ud > 0:
        return ud
    else:
        return ud + p

In [2]:
inv_mod(2,23)

12

In [3]:
3*inv_mod(2,23)%23

13

有限域$F_p$的椭圆曲线$E_p(a,b)$的方程为$$\begin{array}{ccc} 
y^2 = x^3+ax+b(modp) \\
4a^2+27b^2 \neq 0(modp)
\end{array}$$其中，$x,y \in [0, p-1]$
曲线上的点还遵循以下法则：  
1. 无穷远点$\infty$为零元，$\infty+\infty=\infty,P+\infty=\infty$
2. 点$P(x,y)$的负元是$(x, -y(modp))=(x,p-y),P+(-P)=\infty$
3. 点$P(x_1,y_1)$和点$Q(x_2,y_2)$，与点$R(x_3,y_3)$的关系为$$\begin{array}{ccc} 
x_3=l^2-x_1-x_2(modp) \\
y_3=l(x_1-x_3)-y_1(modp)
\end{array}$$其中，$$\left\{ \begin{array}{ll}
l = (y_2-y_1) \times (x_2-x_1)^{-1}(modp) & 如果P \neq Q \\
l = (3x_1^2+a) \times (2y_1)^{-1}(modp)  &  如果P=Q
\end{array} \right.$$

### 展示椭圆曲线上的点

求椭圆曲线$E_29：y^2=x^3+4x+20(mod29)$上的所有点

In [4]:
def show_points(p,a,b):
    return [(x, y) for x in range(p) for y in range(p)
            if (y*y-(x*x*x+a*x+b))%p ==0]

In [5]:
show_points(p=29, a=4, b=20)

[(0, 7),
 (0, 22),
 (1, 5),
 (1, 24),
 (2, 6),
 (2, 23),
 (3, 1),
 (3, 28),
 (4, 10),
 (4, 19),
 (5, 7),
 (5, 22),
 (6, 12),
 (6, 17),
 (8, 10),
 (8, 19),
 (10, 4),
 (10, 25),
 (13, 6),
 (13, 23),
 (14, 6),
 (14, 23),
 (15, 2),
 (15, 27),
 (16, 2),
 (16, 27),
 (17, 10),
 (17, 19),
 (19, 13),
 (19, 16),
 (20, 3),
 (20, 26),
 (24, 7),
 (24, 22),
 (27, 2),
 (27, 27)]

---
已知椭圆曲线$E_5：y^2=x^3+2x+3(mod5)$，试计算$(1,4)+(3,1)$，以及$(1,4) \times 2$

### 倍点运算

In [6]:
def double(x,y,p,a,b):
    l = ((3 * x * x + a) * inv_mod(2 * y,p)) % p
    x3 = (l * l -2 * x) % p
    y3 = (l *(x - x3) - y) % p
    return x3,y3

In [7]:
double(1,4,p=5,a=2,b=3)

(3, 1)

### 加法运算

In [8]:
def add(x1,y1,x2,y2,p,a,b):

    if x1 == x2 and y1 == y2:
        return double(x1,y1,p,a,b)

    l = ((y2 - y1) * inv_mod(x2 - x1,p)) % p
    x3 = (l * l - x1 - x2) % p
    y3 = (l * (x1 - x3) - y1) % p
    return x3,y3

In [9]:
add(1,4,3,1,p=5, a=2, b=3)

(2, 0)

### 数乘运算

第1步，反向获取n的二进制数每一位的集合

In [10]:
def get_bits(n):
    bits = []
    while n != 0:
        bits.append(n & 1)
        n >>= 1
    return bits

In [11]:
from simchain.ecc import Point, CurveFp
# 设置椭圆曲线参数
p, a, b = 29, 4, 20
# 定义一条椭圆曲线
curve = CurveFp(p, a, b)
# 选择椭圆曲线上的一个点
p0 = Point(curve, 3, 1)

In [12]:
# 计算2倍点
p0 * 2

(24,7)

In [13]:
# 计算20倍
p0 * 20

(15,27)

例：已知有限域上的椭圆曲线$E_{37}：y^2 = x^3 -x+3(mod7)$，求点$P(2,3)$的阶

In [14]:
# 设置参数
p, a, b = 37, -1, 3
# 定义椭圆曲线
curve = CurveFp(p, a, b)
# 定义P
p0 = Point(curve, 2, 3)
# 定义-P
_p0 = Point(curve, 2, 34)
p1 = p0
# 从2倍开始计算
n = 2
while p1 != _p0:
    p1 = n*p0
    n += 1
print("n = {0}".format(n))
print("p1 = {0}".format(p1))
print("p1+p0 = {0}".format(p1+p0))

n = 7
p1 = (2,34)
p1+p0 = infinity


由上可知点$P(2,3)$的阶为8，因为$8P=\infty$